## Connect-Four (“4 em Linha”) Game using Minimax with Alpha-Beta Cuts

<img src="https://www.chessprogramming.org/images/thumb/7/79/Connect_4_Board_and_Box.jpg/300px-Connect_4_Board_and_Box.jpg" width="250px" height="250px" align="right">

A board game is characterized by the type of board and tiles, the rules of movement of the pieces (operators/possible moves) and the finishing conditions of the game with the respective score/result.

The game called "Connect Four" in the English language version (“4 em Linha” in the Portuguese version - https://en.wikipedia.org/wiki/Connect_Four) is played on a vertical board of 7x6 squares (i.e., 7 squares wide and 6 squares high), by two players, to which are initially assigned 21 pieces to each (one of the players has white pieces and the other black pieces, or pieces "X" vs pieces "O").

The two players play alternately one of their pieces. The piece to be played is placed on the top of the board and slides either to the base of the board, or in a cell immediately above another one already occupied (see previous figure). The winner will be the player who manages to obtain a line of 4 pieces of its color/symbol horizontally, vertically, or diagonally. If the 42 pieces are played without any player getting a line, the final result will be a draw.

In [ ]:
import random
import time
import numpy as np
from copy import deepcopy

NUM_ROWS = 6
NUM_COLS = 7

class State:
    
    def __init__(self):
        # initialize the board info here and any additional variables
        self.board = np.zeros((NUM_ROWS, NUM_COLS)) # board initial state (all zeros)
        self.column_heights = [NUM_ROWS - 1] * NUM_COLS # useful to keep track of the index in which pieces should be inserted
        self.available_moves = list(range(7)) # list of playable columns (not full)
        self.player = 1
        self.winner = -1 # -1 - no winner (during game); 0 - draw; 1- player 1; 2 - player 2
        
    def move(self, column): 
        # function that performs a move given the column number and returns the new state
        # do not forget to update the available moves list, column heights, pass the turn and check for winners
        state_copy = deepcopy(self)
        
        height = state_copy.column_heights[column]
        state_copy.column_heights[column] = height
        state_copy.board[height][column] = self.player
        
        if height == 0:
            state_copy.available_moves.remove(column)
        else:
            state_copy.column_heights[column] = height - 1
        
        state_copy.update_winner() 
        state_copy.player = 3 - self.player # update player turn
        
        return state_copy
    
    def update_winner(self):
        # function that tests objective and update the winner accordingly
        # should return 1, 2 or 0 (draw)
        for row in range(NUM_ROWS):
            for col in range(NUM_COLS):
                if col < NUM_COLS - 3 and self.check_line(4, self.player, [self.board[row][col], self.board[row][col + 1], self.board[row][col + 2], self.board[row][col + 3]]):
                    self.winner = self.player
                    return self.player
                if row < NUM_ROWS - 3 and self.check_line(4, self.player, [self.board[row][col], self.board[row + 1][col], self.board[row + 2][col], self.board[row + 3][col]]):
                    self.winner = self.player
                    return self.player
                if row < NUM_ROWS - 3 and col < NUM_COLS - 3 and self.check_line(4, self.player, [self.board[row][col], self.board[row + 1][col + 1], self.board[row + 2][col + 2], self.board[row + 3][col + 3]]):
                    self.winner = self.player
                    return self.player
                if row >= 3 and col < NUM_COLS - 3 and self.check_line(4, self.player, [self.board[row][col], self.board[row - 1][col + 1], self.board[row - 2][col + 2], self.board[row - 3][col + 3]]):
                    self.winner = self.player
                    return self.player
        
        if all(height == -1 for height in self.column_heights):
            self.winner = 0
            return 0
        
        return -1

    
    def check_line(self, n, player, values):
        num_pieces = sum(list(map(lambda val: val == player, values)))
        if n == 4:
            return num_pieces == 4
        if n == 3:
            num_empty_spaces = sum(list(map(lambda val: val == 0, values)))
            return num_pieces == 3 and num_empty_spaces == 1
    
    # c1) c2)
    def count_lines(self, n, player):
        num_lines = 0
        for row in range(NUM_ROWS):
            for col in range(NUM_COLS):
                if col < NUM_COLS - 3 and self.check_line(n, player, [self.board[row][col], self.board[row][col+1], self.board[row][col+2], self.board[row][col+3]]):
                    num_lines += 1
                if row < NUM_ROWS - 3 and self.check_line(n, player, [self.board[row][col], self.board[row+1][col], self.board[row+2][col], self.board[row+3][col]]):
                    num_lines += 1
                if row < NUM_ROWS - 3 and col < NUM_COLS - 3 and self.check_line(n, player, [self.board[row][col], self.board[row+1][col+1], self.board[row+2][col+2], self.board[row+3][col+3]]):
                    num_lines += 1
                if row >= 3 and col < NUM_COLS - 3 and self.check_line(n, player, [self.board[row][col], self.board[row-1][col+1], self.board[row-2][col+2], self.board[row-3][col+3]]):
                    num_lines += 1
        return num_lines
    
    # c3)
    def central(self, player):
        central_col = NUM_COLS // 2
        score = 0
        for row in range(NUM_ROWS):
            if self.board[row][central_col] == player:
                score += 2
            if central_col > 0 and self.board[row][central_col - 1] == player:
                score += 1
            if central_col < NUM_COLS - 1 and self.board[row][central_col + 1] == player:
                score += 1
        return score

class ConnectFourGame:
    
    def __init__(self, player_1_ai, player_2_ai):
        self.state = State()
        self.player_1_ai = player_1_ai
        self.player_2_ai = player_2_ai
        
    def start(self, log_moves = False):
        self.state = State()
        while True:
            if self.state.player == 1:
                self.player_1_ai(self)
            else:
                self.player_2_ai(self)
            
            if log_moves:
                print(self.state.board)
            
            if self.state.winner != -1:
                break
        
        if self.state.winner == 0:
            print("End of game! Draw!")
        else:
            print(f"End of game! Player {self.state.winner} wins!")
    
    def run_n_matches(self, n, max_time = 3600, log_moves = False):
        start_time = time.time()
        
        results = [0, 0, 0] # [draws, player 1 victories, player 2 victories]
        
        for _ in range(n):
            self.start(log_moves)
            results[self.state.winner] += 1
            
            if time.time() - start_time > max_time:
                break
     
        print("\n=== Elapsed time: %s seconds ===" % (int(time.time() - start_time)))
        print(f"  Player 1: {results[1]} victories")
        print(f"  Player 2: {results[2]} victories")
        print(f"  Draws: {results[0]} ")
        print("===============================")
               
""" 
    Heuristic functions - e)
"""

def evaluate_f1(state):
    return state.count_lines(4, 1) - state.count_lines(4, 2)

def evaluate_f2(state):
    return 100 * (state.count_lines(4, 1) - state.count_lines(4, 2)) + state.count_lines(3, 1) - state.count_lines(3, 2)

def evaluate_f3(state):
    return 100 * evaluate_f1(state) + state.central(1) - state.central(2)

def evaluate_f4(state):
    return 5 * evaluate_f2(state) + evaluate_f3(state)     

""" 
    Move selection methods
"""
    
def execute_random_move(game):
    move = random.choice(game.state.available_moves)
    game.state = game.state.move(move)
    
def execute_minimax_move(evaluate_func, depth):
    def move(game):
        best_score = float('-inf')
        best_move = None
        for move in game.state.available_moves:
            new_state = game.state.move(move)
            score = minimax(new_state, depth - 1, float('-inf'), float('inf'), False, game.state.player, evaluate_func)
            if score > best_score:
                best_score = score
                best_move = move
        game.state = game.state.move(best_move)
    return move

def minimax(state, depth, alpha, beta, maximizing, player, evaluate_func):
    if depth == 0 or state.winner != -1:
        return evaluate_func(state)
    if maximizing:
        max_eval = float('-inf')
        for move in state.available_moves:
            new_state = state.move(move)
            eval = minimax(new_state, depth - 1, alpha, beta, False, player, evaluate_func)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for move in state.available_moves:
            new_state = state.move(move)
            eval = minimax(new_state, depth - 1, alpha, beta, True, player, evaluate_func)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

# Random vs random
game = ConnectFourGame(execute_random_move, execute_random_move)
game.run_n_matches(10, 120, False)

End of game! Player 2 wins!
End of game! Player 2 wins!
End of game! Player 2 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 2 wins!
End of game! Player 1 wins!
End of game! Player 2 wins!
End of game! Player 2 wins!
End of game! Player 1 wins!

=== Elapsed time: 0 seconds ===
  Player 1: 4 victories
  Player 2: 6 victories
  Draws: 0 


In [ ]:
# Minimax (f1, depth = 2) vs random
game = ConnectFourGame(execute_minimax_move(evaluate_f1, 2), execute_random_move)
game.run_n_matches(10, 120)

End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 2 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!

=== Elapsed time: 2 seconds ===
  Player 1: 9 victories
  Player 2: 1 victories
  Draws: 0 


In [ ]:
# Minimax (f2, depth = 2) vs random
game = ConnectFourGame(execute_minimax_move(evaluate_f2, 2), execute_random_move)
game.run_n_matches(10, 120)

End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!

=== Elapsed time: 6 seconds ===
  Player 1: 10 victories
  Player 2: 0 victories
  Draws: 0 


In [ ]:
# Minimax (f3, depth = 2) vs random
game = ConnectFourGame(execute_minimax_move(evaluate_f3, 2), execute_random_move)
game.run_n_matches(10, 120)

End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!

=== Elapsed time: 3 seconds ===
  Player 1: 10 victories
  Player 2: 0 victories
  Draws: 0 


In [ ]:
# Minimax (f4, depth = 2) vs random
game = ConnectFourGame(execute_minimax_move(evaluate_f4, 2), execute_random_move)
game.run_n_matches(10, 120)

End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!

=== Elapsed time: 7 seconds ===
  Player 1: 10 victories
  Player 2: 0 victories
  Draws: 0 


In [ ]:
# Minimax (f1, depth = 2) vs Minimax (f4, depth = 2)
game = ConnectFourGame(execute_minimax_move(evaluate_f1, 2), execute_minimax_move(evaluate_f4, 2))
game.run_n_matches(5, 120)

End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!

=== Elapsed time: 6 seconds ===
  Player 1: 5 victories
  Player 2: 0 victories
  Draws: 0 


In [ ]:
# Minimax (f4, depth = 2) vs Minimax (f4, depth = 4)
game = ConnectFourGame(execute_minimax_move(evaluate_f4, 2), execute_minimax_move(evaluate_f4, 4))
game.run_n_matches(3, 240, True)

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [2. 0. 0. 1. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [2. 0. 0. 1. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [2. 0. 0. 1. 0. 0. 0.]
 [2. 0. 0. 1. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [2. 0. 0. 1. 0. 0. 0.]
 [2. 0. 0. 1. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [2. 0. 0. 1. 0. 0. 0.]
 [2. 2. 0. 1. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [2. 0. 0. 1. 0. 0. 0.]
 [2. 2. 0.

In [ ]:
###
# Run this code in a Python environment with tkinter installed to play the game. If you don't have tkinter installed, you can install it using pip:
# 
#  pip install tk
#
###

import tkinter as tk
from tkinter import messagebox
import random

NUM_ROWS = 6
NUM_COLS = 7
CELL_SIZE = 100
AI_DELAY = 500
PLAYER_COLORS = {1: "red", 2: "yellow"}

class ConnectFourGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Connect Four")
        self.canvas = tk.Canvas(root, width = NUM_COLS * CELL_SIZE, height = NUM_ROWS * CELL_SIZE)
        self.canvas.pack()
        self.state = State()
        self.draw_board()
        self.canvas.bind("<Button-1>", self.handle_click)
        self.create_menu()
        self.game_mode = "Human vs Human"
        self.ai_player = None

    def create_menu(self):
        menu = tk.Menu(self.root)
        self.root.config(menu = menu)

        game_menu = tk.Menu(menu, tearoff = 0)
        menu.add_cascade(label = "Game", menu = game_menu)
        game_menu.add_command(label = "Human vs Human", command = lambda: self.set_game_mode("Human vs Human"))
        game_menu.add_command(label = "Human vs Random AI", command = lambda: self.set_game_mode("Human vs Random AI"))
        game_menu.add_command(label = "Human vs Minimax AI (Depth 2, Eval F1)", command = lambda: self.set_game_mode("Human vs Minimax AI (Depth 2, Eval F1)", depth = 2, eval_func = evaluate_f1))
        game_menu.add_command(label = "Human vs Minimax AI (Depth 4, Eval F2)", command = lambda: self.set_game_mode("Human vs Minimax AI (Depth 4, Eval F2)", depth = 4, eval_func = evaluate_f2))
        game_menu.add_command(label = "Human vs Minimax AI (Depth 6, Eval F3)", command = lambda: self.set_game_mode("Human vs Minimax AI (Depth 6, Eval F3)", depth = 6, eval_func = evaluate_f3))
        game_menu.add_command(label = "Human vs Minimax AI (Depth 2, Eval F4)", command = lambda: self.set_game_mode("Human vs Minimax AI (Depth 2, Eval F4)", depth = 2, eval_func = evaluate_f4))
        game_menu.add_separator()
        game_menu.add_command(label="Exit", command=self.root.quit)

    def set_game_mode(self, mode, depth=None, eval_func=None):
        self.game_mode = mode
        self.ai_player = None
        if "Minimax AI" in mode:
            self.ai_player = execute_minimax_move(eval_func, depth)
        elif "Random AI" in mode:
            self.ai_player = execute_random_move
        self.reset_game()

    def draw_board(self):
        self.canvas.delete("all")
        for row in range(NUM_ROWS):
            for col in range(NUM_COLS):
                x1 = col * CELL_SIZE
                y1 = row * CELL_SIZE
                x2 = x1 + CELL_SIZE
                y2 = y1 + CELL_SIZE
                self.canvas.create_rectangle(x1, y1, x2, y2, fill = "blue")
                piece = self.state.board[row][col]
                if piece != 0:
                    self.canvas.create_oval(x1 + 5, y1 + 5, x2 - 5, y2 - 5, fill = PLAYER_COLORS[piece])

    def handle_click(self, event):
        col = event.x // CELL_SIZE
        if col in self.state.available_moves:
            self.state = self.state.move(col)
            self.draw_board()
            if self.state.winner != -1:
                self.show_winner()
                return
            if self.game_mode != "Human vs Human" and self.state.player == 2:
                self.root.after(AI_DELAY, self.ai_move)

    def ai_move(self):
        if self.ai_player:
            self.ai_player(self)
            self.draw_board()
            if self.state.winner != -1:
                self.show_winner()

    def show_winner(self):
        if self.state.winner == 0:
            messagebox.showinfo("Game Over", "It's a draw!")
        else:
            messagebox.showinfo("Game Over", f"Player {self.state.winner} wins!")
        self.reset_game()

    def reset_game(self):
        self.state = State()
        self.draw_board()

if __name__ == "__main__":
    root = tk.Tk()
    gui = ConnectFourGUI(root)
    root.mainloop()